In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.cloud import bigquery
import numpy as np
import pandas as pd
import os
import time
import zipfile
from tqdm import tqdm
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "crypto_ethereum" dataset (https://www.kaggle.com/bigquery/ethereum-blockchain)
dataset_ref = client.dataset("crypto_ethereum", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# List all the tables in the "crypto_ethereum" dataset
tables = list(client.list_tables(dataset))

In [ ]:
def query_to_csv(sql, output_path): 
    df = client.query(sql).to_dataframe(progress_bar_type='tqdm_notebook')
    df.to_csv(output_path, mode='a', index=False, header=not os.path.exists(output_path), compression='gzip')

In [ ]:
sql = '''
SELECT nonce, gas_price,from_address,
to_address,block_timestamp,receipt_contract_address, receipt_status
block_number,value, transaction_type FROM 
`bigquery-public-data.crypto_ethereum.transactions` 
WHERE block_number in (12600000, 12601000)
'''
df = client.query(sql).to_dataframe(progress_bar_type='tqdm_notebook')
df.to_csv("Ethereum_transaction.csv")

In [ ]:
df.head()

In [ ]:
#data cleaning
df['timestamp'] = pd.to_datetime(df['block_timestamp'])
df['timestamp'] = df['timestamp'].apply(lambda x: str(x)[:10])
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
df.to_csv("transaction_Ether.csv")

In [ ]:
from __future__ import print_function
import os
import requests
from bs4 import BeautifulSoup
import csv
import time

RESULTS = "results.csv"
URL = "https://etherscan.io/tx/0x2bde6e654eb93c990ae5b50a75ce66ef89ea77fb05836d7f347a8409f141599f"

def getData(sess, page):
    url = URL + page
    print("Retrieving page", page)
    return BeautifulSoup(sess.get(url).text, 'html.parser')

def getPage(sess, page):
    table = getData(sess, str(int(page))).find('table')
    return [[X.text.strip() for X in row.find_all('td')] for row in table.find_all('tr')]

def main():
    resp = requests.get(URL)
    sess = requests.Session()

    with open(RESULTS, 'wb') as f:
        wr = csv.writer(f, quoting=csv.QUOTE_ALL)
        wr.writerow(map(str, "Rank Address Quantity Percentage".split()))
        page = 0
        while True:
            page += 1
            data = getPage(sess, page)

            # Even pages that don't contain the data we're
            # after still contain a table.
            if len(data) < 4:
                break
            else:
                for row in data:
                    wr.writerow(row)
                time.sleep(1)

if __name__ == "__main__":
    main()

In [ ]:
from ethtx import EthTx, EthTxConfig
from ethtx.models.decoded_model import DecodedTransaction


ethtx_config = EthTxConfig(
    mongo_connection_string= ##MongoDB connection string,
    mongo_database= ##MongoDB database,
    etherscan_api_key= ##Etherscan API key,
    web3nodes={
        "mainnet": ##Geth archive node URL,
    },
    default_chain="mainnet",
    etherscan_urls={
        "mainnet": "https://api.etherscan.io/api",
    },
)

ethtx = EthTx.initialize(ethtx_config)
transaction: DecodedTransaction = ethtx.decoders.decode_transaction('0x50051e0a6f216ab9484c2080001c7e12d5138250acee1f4b7c725b8fb6bb922d')